In [1]:
import warnings
warnings.filterwarnings('ignore')

!pip install pydicom
import pandas as pd
import numpy as np
import random
import csv
import os

import pydicom,cv2,pylab
import seaborn as sns
import pydicom as dcm

from skimage import io
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
import keras.backend as K
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator



from matplotlib import pyplot as plt
import matplotlib.patches as patches


     |████████████████████████████████| 35.1MB 119kB/s 


Using TensorFlow backend.


In [2]:
#Mounting drive
from google.colab import drive
drive.mount('./content/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./content/


In [0]:
#Defining project path
project_path = "./content/My Drive/"

In [0]:
#Unzippiong data file
from zipfile import ZipFile
with ZipFile(project_path+'rsna-pneumonia-detection-challenge.zip', 'r') as z:
  z.extractall()

In [0]:
import keras
import tensorflow as tf
from keras import regularizers
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import Model, Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.applications.densenet import DenseNet121
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix,roc_auc_score



In [6]:
os.getcwd()

'/content'

In [45]:
# load and shuffle filenames
folder_test = '/content/stage_2_test_images'
test_filenames = os.listdir(folder_test)
print('n test samples:', len(test_filenames))

n test samples: 3000


In [0]:
random.shuffle(test_filenames)
samples = 100
test_samples = test_filenames[:samples]

MODEL 1 : CHEXNET CLASSIFICATION MODEL 

In [9]:
image_size = 256
# Constant parameters 
input_shape = (image_size, image_size, 3)
num_classes = 1

#Initialize the model

inputs = Input(shape=input_shape)

chexNet = DenseNet121(
          include_top=True
        , input_tensor=inputs
        , weights="./content/My Drive/ChexNet_weight.h5"
        , classes=14
    )
 
chexNet = Model(
          inputs=inputs
        , outputs=chexNet.layers[-2].output
        , name="ChexNet"
    )
 # Weights downloaded from https://github.com/erniechiew/kaggle_rsna_pneumonia_dancingbears/tree/master/classification 

class_model = Sequential()

class_model.add(chexNet)

class_model.add(Dropout(0.6, name="drop_0"))
#model.add(Dense(512, activation = 'relu', name = "dense_0", kernel_regularizer = regularizers.l2(0.01)))
class_model.add(Dense(512, activation = 'relu', name = 'dense_1', kernel_regularizer = regularizers.l2(0.01)))
class_model.add(Dense(256, activation = None, name = 'dense_2', kernel_regularizer = regularizers.l2(0.01)))
class_model.add(LeakyReLU(alpha=0.3))
class_model.add(Dropout(0.6, name="drop_2"))
class_model.add(Dense(1, activation="sigmoid", name="out"))



class_model.summary()















Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ChexNet (Model)              (None, 1024)              7037504   
_________________________________________________________________
drop_0 (Dropout)             (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
drop_2 (Dropout)             (None, 256)               0  

MODEL 2 : CNN SEGMENTATION FOR BOUNDING BOX PREDICTION MODEL

In [10]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
            # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model_bb = keras.Model(inputs=inputs, outputs=outputs)
    return model_bb

# create network and compiler
model_bb = create_network(input_size = 256, channels=32)
model_bb.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 288         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_2 (LeakyReLU)       (None, 256, 256, 32) 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

LOADING CLASSIFICATION MODEL WEIGTHS

In [0]:
class_model.load_weights('/content/content/My Drive/classifier_768_train_aug_adam.h5')

LOADING CNN SEGMENTATION BOUNDING BOX MODEL

In [0]:
model_bb.load_weights('/content/content/My Drive/classifier_bb_256.h5')


- CLASSIFICATION DETECTION MODEL FUNCTION 


In [0]:
from skimage.measure import label,regionprops

#Initializing Dictionary
submission_dict = {}
# loop through testset
def classification_detection(filenames,classification_threshold, detection_threshold):
    negative = 0
    positive = 0
    for filename in filenames:
        img = pydicom.dcmread(os.path.join(folder_test, filename)).pixel_array # reading .dicom images
        # resize image
        img = resize(img, (image_size,image_size), mode='reflect')
        #Normalize
        img = img/255
        #adding trailing dimensions
        img_rgb = np.broadcast_to(img[...,np.newaxis],(1,image_size,image_size,3)) # Converting to 'rgb' for using with ChexNet
        y = class_model.predict(img_rgb) #Prediction using ChexNet Prediction Model
        if y < classification_threshold: # fixing threshold for classification
          predictionString = ''
          negative = negative + 1
          submission_dict[filename] = ''
        else:
          positive = positive + 1
          print(r'positive : {}'.format(positive))
          img = np.broadcast_to(img[...,np.newaxis],(1,image_size,image_size,1)) 
          img = img*255
          #print(img.shape)
          #Predicting bounding box
          pred = model_bb.predict(img)
          patientId = filename
          for pred in pred:
            # resize predicted mask
            pred = resize(pred, (1024, 1024), mode='reflect') #Resizing to 1024 x 1024
            # threshold predicted mask
            comp = pred[:, :, 0] > detection_threshold #fixing threshold for detection
            # apply connected components
            comp = measure.label(comp)
            # apply bounding boxes
            predictionString = ''
            for region in measure.regionprops(comp):
              # retrieve x, y, height and width
              y, x, y2, x2 = region.bbox
              height = y2 - y
              width = x2 - x
              axarr[axidx].add_patch(patches.Rectangle((x,y),width,height,linewidth=2,edgecolor='b',facecolor='none'))
              # proxy for confidence score
              conf = np.mean(pred[y:y+height, x:x+width])
              # add to predictionString
              predictionString += str(conf)  + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) +  ' '
            #add filename and predictionString to dictionary
            patientId = patientId.split('.')[0]
            print(patientId)
            submission_dict[patientId] = predictionString
            # stop if we've got them all
          if len(submission_dict) == len(test_filenames):
            return submission_dict, positive

- DEFINED TWO THRESHOLDS ONE FOR CLASSIFICATION AND ANOTHER FOR DETECTION.  

In [176]:
classification_detection(test_samples,0.4,0.4)

positive : 1
14b6cb8d-0002-4f38-a5b7-7631b56e67df
positive : 2
136a7586-db1b-427d-b375-573b2c602424
positive : 3
134579da-ee59-40a1-a6c1-3d84b9dbbd31
positive : 4
2a31df3c-5131-4622-b63a-be9a058837bd
positive : 5
057a7499-4836-44ef-9490-56b002cd63c2
positive : 6
1f70a977-7222-4952-9ab3-7e1e03bab09c
positive : 7
1b560be4-be73-4976-8d09-06f2dc181701
positive : 8
2d0cb523-9de7-41d3-98a1-d8fcd371e28d
positive : 9
c1330177-9ef2-410c-a3ee-f7c1cabeddd1
positive : 10
2e696db4-4fe7-4de8-a4ae-ac4cf76e2c03
positive : 11
0d2e49e0-8fae-480b-aaa7-eede76cf9d5a
positive : 12
1b42362a-0fec-4327-a3ea-f11b6a05efff
positive : 13
274def79-ae29-4b96-a222-4489e79d54ea
positive : 14
1b3a8b8f-927e-43c2-a0c2-b3f2a996a636
positive : 15
222b5868-4ffe-480d-9441-2e7d793b5049
positive : 16
04170f9f-5d56-4ef7-9960-0afe2b50f778
positive : 17
2d701694-3c54-450e-a236-46cb724f5679
positive : 18
018cb7c3-0ec3-484d-a6a3-bd7c376605d1
positive : 19
2f4dcc33-3ccc-41ca-8b33-9db707891518
positive : 20
2969eca6-35c8-4e4c-8ba7-bb

In [177]:
submission_dict

{'001b0c51-c7b3-45c1-9c17-fa7594cab96e.dcm': '',
 '006be30e-1295-4053-9ffc-fab6b527e7d9.dcm': '',
 '0076e68b-90cf-45da-a839-e67d3131182b.dcm': '',
 '0147a753-be33-4c13-8cd7-9cee88fffcf6.dcm': '',
 '018cb7c3-0ec3-484d-a6a3-bd7c376605d1': '0.78451973 704 192 256 576 0.8693544 384 256 256 512 ',
 '02270587-8e6d-4799-b689-a161db7c5c91.dcm': '',
 '0377791b-ae5b-4abb-b6ae-ac8582ac6f33.dcm': '',
 '03f9e13f-1239-4d82-802d-3ea20731e1db.dcm': '',
 '04170f9f-5d56-4ef7-9960-0afe2b50f778': '0.9326783 192 256 256 512 0.98244494 576 319 256 449 ',
 '045e5500-3dcc-4d9b-8dc2-a85c611dd497': '0.7687206 512 320 320 512 0.9525277 192 448 256 384 ',
 '04994871-0d95-4104-9e60-9573851ee997.dcm': '',
 '04e7a717-567b-4dc3-93ef-6ba4bbdc4a5a.dcm': '',
 '057a7499-4836-44ef-9490-56b002cd63c2': '0.9073762 128 128 320 640 0.97437656 576 128 256 640 ',
 '05b0b6e5-e66c-4bcf-8516-4f7c052651d3.dcm': '',
 '0d2e49e0-8fae-480b-aaa7-eede76cf9d5a': '0.70495516 192 384 256 320 0.9440605 576 448 256 256 ',
 '0ddc2515-b699-4b06-

In [0]:
# save dictionary as csv file
sub = pd.DataFrame.from_dict(submission_dict, orient = 'index')
sub.index.names = ['PatientId']
sub.columns = ['PredictionString']
sub.reset_index()

